In [12]:
import pandas as pd
import numpy as np
import ast
import math
from functools import reduce
import re
import warnings
warnings.filterwarnings('ignore')

In [13]:
data_01 = pd.read_excel('01_код_с_доп_оквэдами.xlsx')
data_10 = pd.read_excel('10_код_с_доп_оквэдами.xlsx')
data_41 = pd.read_excel('41_код_с_доп_оквэдами.xlsx')
data_56 = pd.read_excel('56_код_с_доп_оквэдами.xlsx')
data_86 = pd.read_excel('86_код_с_доп_оквэдами.xlsx')

In [14]:
data = pd.concat([data_01, data_10, data_41, data_56, data_86], ignore_index=True, axis=0)
data = data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', '№ п/п', 'Unnamed: 0.2', 
                          'Дата исключения из реестра', 'Телефон', 'E-mail', 'WWW'])
data.head()

,Наименование / ФИО,Тип субъекта,Категория,ОГРН,ИНН,Основной вид деятельности,Дополнительный вид деятельности,Дополнительный вид деятельности (расшифровка),Регион,Район,Город,Населенный пункт,Вновь созданный,Дата включения в реестр,Наличие лицензий,"Наличие заключенных договоров, контрактов","Производство инновационной, высокотехнологичной продукции",Участие в программах партнерства,Является социальным предприятием,Среднесписочная численность работников за предшествующий календарный год
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""",Юридическое лицо,Среднее предприятие,1112361001103,2361007110,01.11.1 Выращивание зерновых культур,"['01.11.2', '01.11.3', '01.13', '01.13.1', '01...","['Выращивание зернобобовых культур', 'Выращива...",23 - Краснодарский край,Щербиновский р-н,NaN,с Николаевка,Нет,10.08.2017,Да,Нет,Нет,Нет,Нет,95.0
1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОНОВА""",Юридическое лицо,Малое предприятие,1167847206450,7838053444,01.50 Смешанное сельское хозяйство,"['01.11', '01.13', '01.13.1', '01.19', '01.24'...","['Выращивание зерновых (кроме риса), зернобобо...",23 - Краснодарский край,Лабинский р-н,г Лабинск,NaN,Нет,10.08.2017,Да,Нет,Нет,Нет,Нет,93.0
2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОПРОМЫШЛЕННАЯ КОМПАНИ...",Юридическое лицо,Малое предприятие,1022300768555,2304036179,01.2 Выращивание многолетних культур,"['01.21', '01.25', '01.61', '02.10', '02.20', ...","['Выращивание винограда', 'Выращивание прочих ...",23 - Краснодарский край,NaN,г Геленджик,с Пшада,Нет,10.08.2017,Да,Нет,Нет,Нет,Нет,56.0
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОФИРМА КУХАРИВСКАЯ""",Юридическое лицо,Среднее предприятие,1022303857003,2331012177,01.41.11 Разведение молочного крупного рогатог...,"['01.11.1', '01.11.2', '01.11.3', '01.13.3', '...","['Выращивание зерновых культур', 'Выращивание ...",23 - Краснодарский край,Ейский р-н,NaN,с Кухаривка,Нет,10.07.2023,Да,Нет,Нет,Нет,Нет,165.0
4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""БЕЛАГРО""",Юридическое лицо,Среднее предприятие,1042301302713,2303022021,01.50 Смешанное сельское хозяйство,"['01.11', '01.13', '01.2', '01.25', '01.30', '...","['Выращивание зерновых (кроме риса), зернобобо...",23 - Краснодарский край,Белореченский р-н,NaN,с Школьное,Нет,10.08.2019,Нет,Нет,Нет,Нет,Нет,112.0


In [15]:
unique_main_okved = set(re.findall(r'\d{2}[.]\d{1,2}[.]?\d{0,2}[.]?\d{0,2}', data['Основной вид деятельности'].sum()))
unique_dop_okved = set(re.findall(r'\d{2}[.]\d{1,2}[.]?\d{0,2}[.]?\d{0,2}', data['Дополнительный вид деятельности'].sum()))
set_all_okved = sorted(unique_main_okved.union(unique_dop_okved))

In [16]:
def alternative_okved(okved : str) -> str:
    okved_split = okved.split('.')
    if int(okved_split[-1]) == 0:
        return '.'.join(okved_split[:-1])
    else:
        return okved

data_okved = pd.DataFrame(set_all_okved, columns=['ОКВЭД'])
data_okved['Альтернативный ОКВЭД'] = data_okved['ОКВЭД'].apply(lambda x: alternative_okved(x))

In [17]:
data_2014 = pd.read_csv('okved_2014_w_sections.csv', usecols=[3, 4, 6, 8])
data_2014 = data_2014.fillna('')
data_2014['info'] = data_2014['name_section'] + ' ' + data_2014['name_okved'] + ' ' + data_2014['comment']
data_2014 = data_2014.drop(columns=['name_okved', 'comment', 'name_section'])

data_okved_info = data_okved.merge(data_2014, how='left', left_on='Альтернативный ОКВЭД', right_on='native_code')
data_okved_info = data_okved_info.drop(columns=['native_code'])
data_okved_info.head()

,ОКВЭД,Альтернативный ОКВЭД,info
0,01.1,01.1,"Сельское, лесное хозяйство, охота, рыболовство..."
1,01.11,01.11,"Сельское, лесное хозяйство, охота, рыболовство..."
2,01.11.1,01.11.1,"Сельское, лесное хозяйство, охота, рыболовство..."
3,01.11.11,01.11.11,"Сельское, лесное хозяйство, охота, рыболовство..."
4,01.11.12,01.11.12,"Сельское, лесное хозяйство, охота, рыболовство..."


In [18]:
data_okved_info.isna().sum()

ОКВЭД                     0
Альтернативный ОКВЭД      0
info                    104
dtype: int64

In [19]:
lst_okved = reduce(lambda x,y: x + y, [ast.literal_eval(s) for s in data['Дополнительный вид деятельности']])
lst_okved_r = reduce(lambda x,y: x + y, [ast.literal_eval(s) for s in data['Дополнительный вид деятельности (расшифровка)']])
data_dop = pd.DataFrame([lst_okved, lst_okved_r]).T
data_dop = data_dop.drop_duplicates()
data_merge = data_okved_info[data_okved_info['info'].isna()].merge(data_dop.drop_duplicates(0), how='left', left_on='ОКВЭД', right_on=0)


for idx in data_okved_info[data_okved_info['info'].isna()].index:
    data_okved_info.iloc[idx]['info'] = data_merge[data_merge['ОКВЭД'] == data_okved_info.iloc[idx]['ОКВЭД']][1]

In [20]:
data_okved_info.head()

,ОКВЭД,Альтернативный ОКВЭД,info
0,01.1,01.1,"Сельское, лесное хозяйство, охота, рыболовство..."
1,01.11,01.11,"Сельское, лесное хозяйство, охота, рыболовство..."
2,01.11.1,01.11.1,"Сельское, лесное хозяйство, охота, рыболовство..."
3,01.11.11,01.11.11,"Сельское, лесное хозяйство, охота, рыболовство..."
4,01.11.12,01.11.12,"Сельское, лесное хозяйство, охота, рыболовство..."


In [21]:
data_okved_info.isna().sum()

ОКВЭД                   0
Альтернативный ОКВЭД    0
info                    0
dtype: int64

In [13]:
data_okved_info.to_excel('Оквэды с инфо.xlsx')

In [22]:
data.isna().sum()

Наименование / ФИО                                                            0
Тип субъекта                                                                  0
Категория                                                                     0
ОГРН                                                                          0
ИНН                                                                           0
Основной вид деятельности                                                     0
Дополнительный вид деятельности                                               0
Дополнительный вид деятельности (расшифровка)                                 0
Регион                                                                        0
Район                                                                       796
Город                                                                       449
Населенный пункт                                                            976
Вновь созданный                         

In [23]:
mean_num = data['Среднесписочная численность работников за предшествующий календарный год'].values
dict_area = dict(data.groupby(by='Район').mean()['Среднесписочная численность работников за предшествующий календарный год'])
dop_mean = int(data['Среднесписочная численность работников за предшествующий календарный год'].mean())

for i in range(len(mean_num)):
    if math.isnan(mean_num[i]):
        try:
            mean_num[i] = int(dict_area[data.iloc[i]['Район']])
        except KeyError:
            mean_num[i] = dop_mean
            
data['Среднесписочная численность работников за предшествующий календарный год'] = mean_num

In [24]:
for i in data.columns:
    if len(data[i].value_counts()) == 1:
        print(i)

Тип субъекта
Регион
Вновь созданный
Наличие заключенных договоров, контрактов
Производство инновационной, высокотехнологичной продукции
Участие в программах партнерства


In [25]:
data = data.fillna('')

data['Адрес'] = data['Район'] + ', ' + data['Город'] + ', ' + data['Населенный пункт']
data = data.drop(columns=['Регион', 'Район', 'Город', 'Населенный пункт', 'Тип субъекта', 
                          'Вновь созданный', 'Наличие заключенных договоров, контрактов',
                         'Производство инновационной, высокотехнологичной продукции', 'Участие в программах партнерства'])
data.head()

,Наименование / ФИО,Категория,ОГРН,ИНН,Основной вид деятельности,Дополнительный вид деятельности,Дополнительный вид деятельности (расшифровка),Дата включения в реестр,Наличие лицензий,Является социальным предприятием,Среднесписочная численность работников за предшествующий календарный год,Адрес
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""",Среднее предприятие,1112361001103,2361007110,01.11.1 Выращивание зерновых культур,"['01.11.2', '01.11.3', '01.13', '01.13.1', '01...","['Выращивание зернобобовых культур', 'Выращива...",10.08.2017,Да,Нет,95.0,"Щербиновский р-н, , с Николаевка"
1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОНОВА""",Малое предприятие,1167847206450,7838053444,01.50 Смешанное сельское хозяйство,"['01.11', '01.13', '01.13.1', '01.19', '01.24'...","['Выращивание зерновых (кроме риса), зернобобо...",10.08.2017,Да,Нет,93.0,"Лабинский р-н, г Лабинск,"
2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОПРОМЫШЛЕННАЯ КОМПАНИ...",Малое предприятие,1022300768555,2304036179,01.2 Выращивание многолетних культур,"['01.21', '01.25', '01.61', '02.10', '02.20', ...","['Выращивание винограда', 'Выращивание прочих ...",10.08.2017,Да,Нет,56.0,", г Геленджик, с Пшада"
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОФИРМА КУХАРИВСКАЯ""",Среднее предприятие,1022303857003,2331012177,01.41.11 Разведение молочного крупного рогатог...,"['01.11.1', '01.11.2', '01.11.3', '01.13.3', '...","['Выращивание зерновых культур', 'Выращивание ...",10.07.2023,Да,Нет,165.0,"Ейский р-н, , с Кухаривка"
4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""БЕЛАГРО""",Среднее предприятие,1042301302713,2303022021,01.50 Смешанное сельское хозяйство,"['01.11', '01.13', '01.2', '01.25', '01.30', '...","['Выращивание зерновых (кроме риса), зернобобо...",10.08.2019,Нет,Нет,112.0,"Белореченский р-н, , с Школьное"


In [27]:
data['Наличие лицензий'] = data['Наличие лицензий'].map({'Да': 1, 'Нет': 0})
data['Является социальным предприятием'] = data['Является социальным предприятием'].map({'Да': 1, 'Нет': 0})
data['Категория'] = data['Категория'].map({'Среднее предприятие': 1, 'Малое предприятие': 0})
data.head()

,Наименование / ФИО,Категория,ОГРН,ИНН,Основной вид деятельности,Дополнительный вид деятельности,Дополнительный вид деятельности (расшифровка),Дата включения в реестр,Наличие лицензий,Является социальным предприятием,Среднесписочная численность работников за предшествующий календарный год,Адрес
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""",1,1112361001103,2361007110,01.11.1 Выращивание зерновых культур,"['01.11.2', '01.11.3', '01.13', '01.13.1', '01...","['Выращивание зернобобовых культур', 'Выращива...",10.08.2017,1,0,95.0,"Щербиновский р-н, , с Николаевка"
1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОНОВА""",0,1167847206450,7838053444,01.50 Смешанное сельское хозяйство,"['01.11', '01.13', '01.13.1', '01.19', '01.24'...","['Выращивание зерновых (кроме риса), зернобобо...",10.08.2017,1,0,93.0,"Лабинский р-н, г Лабинск,"
2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОПРОМЫШЛЕННАЯ КОМПАНИ...",0,1022300768555,2304036179,01.2 Выращивание многолетних культур,"['01.21', '01.25', '01.61', '02.10', '02.20', ...","['Выращивание винограда', 'Выращивание прочих ...",10.08.2017,1,0,56.0,", г Геленджик, с Пшада"
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОФИРМА КУХАРИВСКАЯ""",1,1022303857003,2331012177,01.41.11 Разведение молочного крупного рогатог...,"['01.11.1', '01.11.2', '01.11.3', '01.13.3', '...","['Выращивание зерновых культур', 'Выращивание ...",10.07.2023,1,0,165.0,"Ейский р-н, , с Кухаривка"
4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""БЕЛАГРО""",1,1042301302713,2303022021,01.50 Смешанное сельское хозяйство,"['01.11', '01.13', '01.2', '01.25', '01.30', '...","['Выращивание зерновых (кроме риса), зернобобо...",10.08.2019,0,0,112.0,"Белореченский р-н, , с Школьное"


In [29]:
data.isna().sum()

Наименование / ФИО                                                          0
Категория                                                                   0
ОГРН                                                                        0
ИНН                                                                         0
Основной вид деятельности                                                   0
Дополнительный вид деятельности                                             0
Дополнительный вид деятельности (расшифровка)                               0
Дата включения в реестр                                                     0
Наличие лицензий                                                            0
Является социальным предприятием                                            0
Среднесписочная численность работников за предшествующий календарный год    0
Адрес                                                                       0
dtype: int64

In [30]:
data.to_excel('Датасет.xlsx')